In [1]:
import pandas as pd
import numpy

In [2]:
import codecs
import os
with codecs.open('customer_data.csv', 'r') as file:
    BLOCKSIZE = 512
    if not os.path.exists('customer_data(utf-8).csv'):
        with codecs.open('customer_data(utf-8).csv', "w", "utf-8") as targetFile:
            while True:
                contents = file.read(BLOCKSIZE)
                if not contents:
                    break
                targetFile.write(contents)

FileNotFoundError: [Errno 2] No such file or directory: 'customer_data.csv'

In [2]:
dtype = {
    '交易id': numpy.str,
    '資料日期': numpy.str,
    '資料時間': numpy.str,
    '餐別帶': numpy.str,
    '縣市別': numpy.str,
    '店舖代號': numpy.uint32,
    '主商圈': numpy.str,
    '品號-品名稱': numpy.str,
    '群號-群名稱': numpy.str,
    '單品名稱': numpy.str,
    '銷售數量': numpy.uint16,
    '銷售單價': numpy.float,
    '交易金額': numpy.float
}
USE_COLUMNS = ['交易id', '資料日期', '資料時間', '餐別帶', '縣市別', '店舖代號', '主商圈', '品號-品名稱',
       '群號-群名稱', '單品名稱', '銷售數量', '銷售單價', '交易金額']
PARSE_DATES = {
    '資料日期與時間': [
        '資料日期',
        '資料時間'
    ]
}

In [3]:
file = pd.read_csv('customer_data(utf-8).csv',
                   index_col=1,
                   nrows=100000,
                   usecols=USE_COLUMNS,
                   dtype=dtype,
                   parse_dates=PARSE_DATES,
        )

In [4]:
file = file.dropna()

In [5]:
file.shape

(99981, 11)

In [6]:
def get_copurchase_df(df):
    count_size = df.index.unique()
    size_df = df.groupby('交易id').size()
    size_df = size_df[size_df < 2]
    return df.drop(list(size_df.index), axis=0)

In [7]:
data = get_copurchase_df(file)

In [8]:
def aggregate(df):
    dic = {}
    for index, row in df.iterrows():
        if index not in dic:
            dic[index] = []
        dic[index].append({
            'name': row['單品名稱'],
            'amount': row['銷售單價']
        })
    dic = { index : value for index, value in dic.items() if len(value) > 1}
    return dic

In [9]:
purchase_list = aggregate(data)

In [10]:
len(purchase_list)

23288

In [11]:
purchase_list

{'01637920171201184556000112534201': [{'name': '（新）銷售用購物袋１８號袋', 'amount': 1.0},
  {'name': '瑞穗低脂鮮乳９３０ｍｌ', 'amount': 87.0},
  {'name': '鮮奶油布蕾', 'amount': 25.0},
  {'name': '統一大布丁（雞蛋口味）', 'amount': 20.0},
  {'name': '鮮奶茶', 'amount': 40.0}],
 '01614720171201163042000114405402': [{'name': '飲冰室茶集綠奶茶', 'amount': 25.0},
  {'name': '雙薯泥燻雞起司', 'amount': 39.0}],
 '01384020171201130558000119295301': [{'name': '沙茶牛肉炒麵', 'amount': 69.0},
  {'name': 'ＦＩＮ健康補給飲料９７５ｍｌ', 'amount': 35.0}],
 '01475120171201153506000111697202': [{'name': '冷泡茶台茶１２號紅茶', 'amount': 25.0},
  {'name': '草莓天使蛋糕', 'amount': 28.0}],
 '01564720171201180924000113751602': [{'name': '多喝水２Ｌ', 'amount': 35.0},
  {'name': '愛之味分解茶', 'amount': 35.0}],
 '01614720171201054959000114383602': [{'name': '光泉無加糖鮮豆漿', 'amount': 20.0},
  {'name': '光泉米漿', 'amount': 20.0},
  {'name': '起司派對三明治', 'amount': 35.0},
  {'name': '肉鬆細卷', 'amount': 23.0}],
 '01614720171201004542000114379002': [{'name': '卡迪那牛排卡大包', 'amount': 35.0},
  {'name': '波的多洋芋片蚵仔煎口味', 'amou

# Network Analysis

In [12]:
from itertools import filterfalse
def convert(purchase_list, support):
    def find_edges_in_list(itemsets):
        from itertools import combinations
        result = []
        return combinations(itemsets, 2)
    result = {}
    nodes = set()
    for key, itemsets in purchase_list.items():
        if len(itemsets) > 1:
            for edge_dict_tuple in find_edges_in_list(itemsets):
                edge = tuple([dic['name'] for dic in edge_dict_tuple])
                weight = sum([dic['amount'] for dic in edge_dict_tuple])
                if edge in result:
                    result[edge]['count'] += 1
                else:
                    result[edge] = {}
                    result[edge]['count'] = 1
                    result[edge]['weight'] = weight
    for key in list(result.keys()):
        if result[key]['count'] < support:
            del result[key]
    for items in result.keys():
        for item in items:
            if item not in nodes:
                nodes.add(item)
    return (nodes, result)

In [13]:
nodes, edges_dict = convert(purchase_list, 5)

In [61]:
print('Node number: {}\nEdge number: {}'.format(len(nodes), len(edges_dict)))

Node number: 400
Edge number: 1029


In [16]:
import igraph

In [17]:
g = igraph.Graph()

In [18]:
for node in nodes:
    g.add_vertex(node)

In [19]:
len(g.vs)

400

In [20]:
for edge, attrs in edges_dict.items():
    weight = 1/attrs['weight'] * attrs['count'] if attrs['weight'] > 0 else 0
    g.add_edge(edge[0], edge[1], weight=weight)

In [21]:
'（新）銷售用購物袋１８號袋' in nodes

True

In [22]:
len(g.es)

1029

In [30]:
g.vs.find(name='紅標純米料理米酒')

igraph.Vertex(<igraph.Graph object at 0x1112b0318>, 4, {'name': '紅標純米料理米酒'})

In [24]:
g.es[1]

igraph.Edge(<igraph.Graph object at 0x1112b0318>, 1, {'weight': 0.12195121951219512})

In [25]:
g.simplify(combine_edges={ "weight": "sum" })

In [39]:
communities = g.community_fastgreedy('weight')

In [62]:
len(communities)

TypeError: object of type 'VertexDendrogram' has no len()

In [73]:
cluster = communities.as_clustering()

In [74]:
len(cluster)

51

In [75]:
print("==================")
for comm in cluster:
    comm_name = [ g.vs[index]['name'] for index in comm]
    print(" ".join(comm_name))
    print("==================")

聯合報 自由時報 中國時報 蘋果日報
黃金厚切魚板 王子麵 手工高麗菜捲 日式黑輪 究極味付蛋 千層玉子燒 鮮香菇 蟹肉糰子 讚岐烏龍麵 筊白筍 白玉蘿蔔 杏鮑菇 特級花枝丸 海鮮魚卵棒 關東煮本舖拉麵 旗魚黑輪 關東煮本舖冬粉 野菜多多魚餅 黃金魚豆腐
地方查核稅款 勞保費代收 富邦信用卡 欣中天然氣 台灣自來水 遠東信用卡 欣林瓦斯費 台中二段停 國泰世華卡 國民年金代 台新信用卡 永豐信用卡 玉山信用卡 台灣大哥大 代收手續費４ 中華電信 遠傳電信 勞工退休金 聯邦信用卡 彰化縣停車 健保費代收 澳盛信用卡 中信有線代 花旗信用卡 台灣電力 合庫瓦斯費 台新公用代 亞太電信代 台灣星電信 振道有線
台鹽海洋鹼性離子水 純喫茶綠茶６５０ｍｌ 蕃薯（１５元） 統一多多活菌發酵乳原味 麥香紅茶ＴＰ３７５ 藍山咖啡 貝納頌經典拿鐵 養樂多３００ＬＩＧＨＴ活菌發酵 超涼口香糖（袋） ＥＣ代收一 及第豬肉熟水餃 肉絲炒炊粉 麥香奶茶（罐裝） 阿Ｑ韓式泡菜桶麵 安全裝置火石打火機 天然水２．２Ｌ 麥香綠茶ＴＰ３００ 優格軟糖（Ｐｅａｃｈ） 優格軟糖（Ｏｒａｎｇｅ） 御茶園特上檸檬茶 波蜜果菜汁 光泉巧克力牛乳 Ｃｒｅａｍ－Ｏ黑巧克力三明治餅 蔥爆牛肉燴飯 玉米濃湯 ＡＢ優酪乳原味 咖啡廣場鋁箔包 ＡＷ極酷嗆辣口香糖 統一脆麵 阿Ｑ桶麵（紅椒牛肉） 義美古早傳統無糖豆奶 純喫茶無糖綠茶６５０ｍｌ 頑皮滷蛋－原味 虎皮蛋糕捲 （新）４５號銷售用購物袋 養樂多（５入） 麥香阿薩姆奶茶 泡沫紅茶ＴＰ３００ 茶裏王英式紅茶 雲絲頓紅１０毫克香煙 純喫茶紅茶６５０ｍｌ 蝦皮取件Ｃ 七星風藍４毫克硬盒 七星天藍７毫克硬盒 養樂多 茶裏王日式綠茶 義美錫蘭紅茶 ＡＷ極酷嗆涼紫冰野莓口香糖 商店街取件 巧克力捲 蘋果西打 茶裏王白毫烏龍茶 奶酥麵包 伊藤園蘋果紅茶 可樂果—酷辣口味 七星１０毫克軟包 全家紅豆麵包 來一客牛肉蔬菜杯麵 肉鬆起酥麵包 純喫茶無糖綠茶 頂級鮮奶優格－１００％蜂蜜 冷泡茶冷萃綠茶無糖 芝麻米漿 義美奶茶 蜂蜜水 金牌台灣啤酒 葡萄冰茶 白蘭氏活顏馥莓飲 簡單點無加糖優酪乳 保力達蠻牛維他命Ｂ飲料 三星蔥抓餅 麥香紅茶ＴＰ３００ 海苔肉鬆沙拉麵包 墨西哥葡萄奶酥 純喫茶綠茶 番薯（３５元） 義美全脂鮮乳２３６ｍｌ 統一ＰＨ９鹼性離子水 蕃薯（２５元） 熱拿鐵小杯 菠蘿

In [83]:
cluster.modularity

0.7450238439465181